# IMPORT LIBRARIES

In [1]:
import MetaTrader5 as mt5

import pandas as pd

from datetime import datetime

import pytz

In [11]:
import numpy as np

# CONFIG

In [2]:
pd.set_option('display.float_format', '{:.5f}'.format)

In [3]:
gc_o_TIME_ZONE = pytz.timezone("Etc/UTC")
gc_dt_FROM = datetime(2022, 3, 1, tzinfo=gc_o_TIME_ZONE)
gc_dt_TO = datetime(2022, 7, 1, tzinfo=gc_o_TIME_ZONE)

# UDFS

In [4]:
def dfFetchSampleDataFromMt(p_sSymbolName):
    aOhlSample = mt5.copy_rates_range(
        p_sSymbolName,
        mt5.TIMEFRAME_M15,
        gc_dt_FROM, 
        gc_dt_TO
    )

    dfToReturn = pd.DataFrame(aOhlSample)
    if 'time' in list(dfToReturn.columns):
        dfToReturn.set_index('time', inplace = True)
    else:
        return dfFetchSampleDataFromMt(p_sSymbolName)
    
    return dfToReturn

In [5]:
def dfCalculateReturn(p_dfToSplit):
    p_dfToSplit['RETURN'] =  (p_dfToSplit['close']-p_dfToSplit['open'])/p_dfToSplit['open']
    return p_dfToSplit

# MAIN

In [6]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
    
tplSymbols = mt5.symbols_get()
dfSymbols = pd.DataFrame(tplSymbols, columns = tplSymbols[0]._asdict().keys())

In [7]:
def aGetUniqueCategories(tplSymbols):
    aCategories = []
    for i in range(0, len(tplSymbols)):
        sCategory = '\\'.join(tplSymbols[i]._asdict()['path'].split('\\')[:-1]) +'\\'
        if sCategory not in aCategories:
            aCategories.append(sCategory)
    return aCategories

In [8]:
aCategories = aGetUniqueCategories(tplSymbols)

sCategory = 'Stocks\\Germany\\Banking & Investment Services\\Banks\\'
dfFilteredSymbols = dfSymbols[dfSymbols['path'].str.contains(sCategory, regex=False) == True]

dfOhlc = pd.DataFrame()

for iIndex, srsRow in dfFilteredSymbols.iterrows():
    sSymbolName = dfFilteredSymbols.loc[iIndex, 'name']

    dfOhlcSample =  dfFetchSampleDataFromMt(sSymbolName)
    if len(dfOhlcSample) > 1000:
        dfOhlcSample = dfCalculateReturn(dfOhlcSample)
        dfOhlcSample['SYMBOL_NAME'] =  sSymbolName
        dfOhlc = dfOhlc.append(dfOhlcSample)

# OUTPUT DATASET

In [9]:
dfOhlc['RETURN_ABS'] = abs(dfOhlc['RETURN'])

ixNegative = dfOhlc[dfOhlc['RETURN'] < 0].index
ixPositive = dfOhlc[dfOhlc['RETURN'] >= 0].index
dfOhlc.loc[ixNegative, 'RETURN_SIGN'] = 'negative'
dfOhlc.loc[ixPositive, 'RETURN_SIGN'] = 'positive'

## Return Prediction

In [10]:
dfOhlcCopy = dfOhlc.copy()

aReturnSigns = ['negative', 'positive']
aForwardTimeSteps = list(range(0, 5))
aExchangeRates = list(dfOhlcCopy['SYMBOL_NAME'].unique())


ixCols = pd.MultiIndex.from_product(
    [aReturnSigns, aForwardTimeSteps, aExchangeRates],
    names=['RETURN_SIGNS', 'FORWARD_TIME_STEPS', 'EXCHANGE_RATES']
)

ixRows = dfOhlcCopy.index.unique().sort_values()


dfReturnPrediction = pd.DataFrame(index = ixRows ,columns = ixCols)


for ix in ixCols:
    sReturnSign  = ix[0]
    iForwardTimeStep  = ix[1]
    sExchangeRate = ix[2]
    
    
    df = dfOhlcCopy[(dfOhlcCopy['RETURN_SIGN'] == sReturnSign) & (dfOhlcCopy['SYMBOL_NAME'] == sExchangeRate)]
    srs = df['RETURN_ABS'].shift(-iForwardTimeStep)
    
    dfReturnPrediction.loc[srs.index, (sReturnSign, iForwardTimeStep, sExchangeRate)] = srs.values
    
dfReturnPrediction.fillna(0, inplace = True)
dfReturnPrediction

RETURN_SIGNS       negative                                                   \
FORWARD_TIME_STEPS        0                                        1           
EXCHANGE_RATES      ARLG.DE CBKG.DE DBKGn.DE FTKn.DE PBBG.DE ARLG.DE CBKG.DE   
time                                                                           
1646128800          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1646129700          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1646130600          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1646131500          0.00147 0.01339  0.00542 0.00437 0.00699 0.00368 0.01223   
1646132400          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
...                     ...     ...      ...     ...     ...     ...     ...   
1656609300          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1656610200          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1656611100          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1656612000          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   
1656612900          0.00000 0.00000  0.00000 0.00000 0.00000 0.00000 0.00000   

RETURN_SIGNS                                 ... positive                   \
FORWARD_TIME_STEPS                           ...        3                    
EXCHANGE_RATES     DBKGn.DE FTKn.DE PBBG.DE  ...  ARLG.DE CBKG.DE DBKGn.DE   
time                                         ...                             
1646128800          0.00000 0.00000 0.00000  ...  0.00074 0.00136  0.00182   
1646129700          0.00000 0.00000 0.00000  ...  0.00074 0.00825  0.00462   
1646130600          0.00000 0.00000 0.00000  ...  0.00148 0.00139  0.00000   
1646131500          0.01367 0.00382 0.01605  ...  0.00000 0.00000  0.00000   
1646132400          0.00000 0.00000 0.00000  ...  0.00371 0.01662  0.00093   
...                     ...     ...     ...  ...      ...     ...      ...   
1656609300          0.00000 0.00000 0.00000  ...  0.00000 0.00759  0.00606   
1656610200          0.00000 0.00000 0.00000  ...  0.00000 0.00452  0.00120   
1656611100          0.00000 0.00000 0.00000  ...  0.00000 0.00000  0.00000   
1656612000          0.00000 0.00000 0.00000  ...  0.00000 0.00000  0.00000   
1656612900          0.00000 0.00000 0.00000  ...  0.00000 0.00000  0.00000   

RETURN_SIGNS                                                                 
FORWARD_TIME_STEPS                       4                                   
EXCHANGE_RATES     FTKn.DE PBBG.DE ARLG.DE CBKG.DE DBKGn.DE FTKn.DE PBBG.DE  
time                                                                         
1646128800         0.00659 0.00100 0.00074 0.00825  0.00462 0.00055 0.00306  
1646129700         0.00055 0.00306 0.00148 0.00139  0.00000 0.00328 0.00203  
1646130600         0.00328 0.00203 0.00371 0.01662  0.00093 0.00220 0.00102  
1646131500         0.00000 0.00000 0.00000 0.00000  0.00000 0.00000 0.00000  
1646132400         0.00220 0.00102 0.00074 0.00282  0.00093 0.00384 0.00203  
...                    ...     ...     ...     ...      ...     ...     ...  
1656609300         0.00561 0.00354 0.00000 0.00452  0.00120 0.00780 0.00353  
1656610200         0.00780 0.00353 0.00000 0.00000  0.00000 0.00000 0.00000  
1656611100         0.00000 0.00000 0.00000 0.00000  0.00000 0.00000 0.00000  
1656612000         0.00000 0.00000 0.00000 0.00000  0.00000 0.00000 0.00000  
1656612900         0.00000 0.00000 0.00000 0.00000  0.00000 0.00000 0.00000  

[2924 rows x 50 columns]